In [3]:
""" 
Created on Wed Jan 23 2019
 
MTS Program that graphs difference in mahalanobis distances, as well as an MTS 
 
@author: Taehee 
""" 
# DB연결
from __future__ import print_function
import os
import pyodbc  
# 수치계산
import numpy as np 
import matplotlib as plt 
import pandas as pd
from sklearn import preprocessing

# 버전확인
import sys
import PyQt5
print(sys.version) 

#--------------------------------------------------------------------------------#

In [37]:
def remakeNomarlMTS():
    # mssql 접속 2개 만들기 
    conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                          "Server=MDPLP1546701\\SQLEXPRESS;"
                          "Database=ROLLCURLING_MONITORING;"
                          "Trusted_Connection=yes;")

    #connection에서 cursor 생 (cursor는 fetch 동작 관리)
    cursor = conn.cursor()
    # OS와 맞는 DB설치할 것
    cursor.execute("SELECT VAR_AWEIGHT, VAR_CWEIGHT, VAR_TQU_CTIME, VAR_RPM_BIAS, VAR_H_MINUS_CL FROM dbo.MTS_BASE_DATA ; ")
        # VALUE01 : 축력하중(kgf)
        # VALUE02 : 컬링하중(kgf)
        # VALUE03 : 토크상한 누적시간
        # VALUE04 : RPM 편차(rpm)
        # VALUE05 : 높이측정(mm) (사용x)
        # VALUE06 : 완료위치(mm) (사용x)
        # VALUE07 : 높이측정-완료위치(mm)
        # VALUE08 : 단차 측정(mm) (SAM2라인 사용X)

    row = cursor.fetchall()
    # n, k, factor별 mean, std 확인
    OKdata=np.array(row)
    n = OKdata.shape[0]
    k = OKdata.shape[1]
    print("제품 수(n):",n,",","factor 수(k):",k)
    
    
    OKdata_factor_mean=[]
    OKdata_factor_std=[]
    
    
    for i in range(k):
        OKdata_factor_mean.append(float(OKdata.mean(axis=0)[i]))
        OKdata_factor_std.append(float(OKdata.std(axis=0)[i]))

    # 데이터 정규화(Z)
    scaler = preprocessing.StandardScaler()
    scaled_df_ok = scaler.fit_transform(OKdata)

    # 상관계수(R) 구하기
    pd_scaled_df_ok = pd.DataFrame(scaled_df_ok)
    OKdata_corr = pd_scaled_df_ok.corr()
    OKdata_corr
    OKdata_corr.shape

    # 상관계수의 역함수(R^-1)
    cor =np.linalg.inv(OKdata_corr)

    conn.close()
    
    
    # 파일로 저장
    OKdata_factor_mean=pd.DataFrame(OKdata_factor_mean).T
    OKdata_factor_std=pd.DataFrame(OKdata_factor_std).T
    cor=pd.DataFrame(cor)
    
    os.chdir("C:/Users/mando/Desktop/MTS_system/base_data")
    OKdata_factor_mean.to_csv("OKdata_factor_mean.csv")
    OKdata_factor_std.to_csv("OKdata_factor_std.csv")
    cor.to_csv("cor.csv")

In [38]:
remakeNomarlMTS()

제품 수(n): 26291 , factor 수(k): 5


C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
